In [1]:
import sys
import xlsxwriter
import pandas as pd
import sys
import psycopg2
import os
from dotenv import load_dotenv
from datetime import date

today = date.today()
load_dotenv()

#configurar la conexion a la base de datos
DB_USERNAME = os.getenv('DB_USERNAME')
DB_DATABASE = os.getenv('DB_DATABASE')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_PORT = os.getenv('DB_PORT')
DB_HOST=os.getenv('DB_HOST')
# Conectar a DB
# Conectar a PostgreSQL
try:
    cnx = psycopg2.connect(
        user=DB_USERNAME,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT,
        database=DB_DATABASE
    )
    print("Conexión exitosa")
except psycopg2.Error as e:
    print("Ocurrió un error al conectar a la base de datos:", e)


Conexión exitosa


In [17]:
reactivos=pd.read_sql("select * from reactivos where section = 'encuesta_verde' order by orden" ,cnx)
opciones=pd.read_sql("select * from options where reactivo like '%vr%' order by orden" ,cnx)

c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [38]:
for i in opciones.reactivo.unique():
    log=f' {i}: '
    ops=opciones.loc[opciones['reactivo']==i]
    for j in range(len(ops)):
        log=log+str(ops['clave'].values[j])+': '+ops['descripcion'].values[j]+', '
    print(log)


 vr6: 1: Eficiencia energética, 2: Conservación aguas superficiales y aguas subterráneas, 27: Energías renovables, 5: Compostaje, 25: Fomento del transporte público, 26: Uso de Biocombustibles, 29: instalación y mantenimiento de sistemas de energía solar, 3: Gestión del agua, 4: Tratamiento de aguas residuales, 6: Gestión de residuos, 7: Reciclaje de residuos, 8: Divulgación ambiental, 9: Capacitación ambiental, 10: Investigación y Desarrollo ambiental, 11: Agricultura y pesca sostenible, 12: Conservación ambiental, 13: Consultoría en sostenibilidad, 14: Ecoturismo, 15: Evaluación ambiental, 16: Gestión de proyectos sustentables, 17: Gestión de riesgos naturales, 18: Impacto ambiental, 19: Minería, 20: Ordenamiento Territorial, 21: Participación en iniciativas comunitarias ambientales, 22: Diseño de edificios inteligentes, 23: Diseño de materiales ecológicos, 24: Reciclado de materiales y/o residuos de la construcción, 28: Gestión energética, 30: Otro (especifique), 
 vr7: 49: Otra esp

In [28]:
pd.read_sql("select * from respuestas_verdes where vr1 is not null" ,cnx).to_excel('encuesta_Verde.xlsx')

In [26]:
reactivos[['orden','clave','description','type']].to_excel('respuestas_opciones_clave.xlsx')

In [22]:
opciones.loc[opciones['reactivo']=='vr1']

Empty DataFrame
Columns: [id, reactivo, clave, descripcion, updated_at, orden, help_info, update_rules]
Index: []

In [107]:
empresas=pd.read_sql("select * from empresas", cnx)

c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [108]:
encuestas20= pd.read_sql("select * from respuestas20", cnx)

c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [109]:
encuestas16= pd.read_sql("select * from respuestas16", cnx)

c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [110]:
import re
def normalize(old):
    """
    Removes common accent characters, lower form.
    Uses: regex.
    """
    if not isinstance(old, str):
        return old
    new = old.lower()
    new = re.sub(r'[àáâãäå]', 'a', new)
    new = re.sub(r'[èéêë]', 'e', new)
    new = re.sub(r'[ìíîï]', 'i', new)
    new = re.sub(r'[òóôõö]', 'o', new)
    new = re.sub(r'[ùúûü]', 'u', new)
    new = re.sub(r'[,]', '', new)
    new = re.sub(r'["]', '', new)
    return new

In [111]:
normalize('Atención,2')

'atencion2'

In [112]:
empresas['nombre']=empresas['nombre'].fillna(' ')
encuestas16['ncr2']=encuestas16['ncr2'].fillna(' ')
encuestas20['ncr2']=encuestas20['ncr2'].fillna(' ')

In [113]:
empresas['nombre_normalized']= empresas['nombre'].apply(lambda x: normalize(x))
encuestas20['ncr2_normalized']= encuestas20['ncr2'].apply(lambda x: normalize(x))
encuestas16['ncr2_normalized']= encuestas16['ncr2'].apply(lambda x: normalize(x))

In [65]:
empresas.loc[(empresas['registro'].notna())&(empresas['registro']!=-1)]

id usuario                                             nombre  giro  \
1        7      24                                      grupo promesa  None   
2     4529      25                                              indat  None   
3       21      25                                laboratorios carnot  None   
4       32      23                                            bicimex  None   
5       20      24                                      grupo grabach  None   
...    ...     ...                                                ...   ...   
2495  4843      23                                     solarig mexico  None   
2497  4856      17  direccion de desarrollo agropecuario y rural c...  None   
2498  4854      24                                     mettler toledo  None   
2500  4816      17      unam colegio ciencias y humanidades naucalpan  None   
2502   722      24  instituto nacional de ecologia y cambio climat...  None   

     clave_giro                                    giro_especifico  \
1            16  611698 Otros servicios educativos proporcionad...   
2            12                      541810 Agencias de publicidad   
3             5  325412 Fabricación de preparaciones farmacéuticas   
4             7                              Comercio al por menor   
5            12                      541810 Agencias de publicidad   
...         ...                                                ...   
2495          3  221113 Generación de electricidad a partir de ...   
2497         20  931310 Regulación y fomento del desarrollo eco...   
2498          6  435 Comercio al por mayor de maquinaria, equip...   
2500         16  611162 Escuelas de educación media superior de...   
2502         12  541712 Servicios de investigación científica y...   

                             nota created_at                updated_at  \
1                   Empresa verde        NaT 2025-06-19 19:33:23+00:00   
2                            None        NaT 2025-08-14 20:19:27+00:00   
3             Empresa sustentable        NaT 2025-01-06 21:28:04+00:00   
4                            None        NaT                       NaT   
5                            None        NaT                       NaT   
...                           ...        ...                       ...   
2495  Empresa verde y sustentable        NaT                       NaT   
2497                         None        NaT                       NaT   
2498                         None        NaT                       NaT   
2500                         None        NaT 2025-09-08 22:36:00+00:00   
2502            Institución verde        NaT 2025-09-09 16:08:38+00:00   

      registro  sector  
1     161259.0       2  
2     163557.0       2  
3     161278.0       2  
4     161289.0       2  
5     161277.0       2  
...        ...     ...  
2495  164139.0       2  
2497  164179.0       1  
2498  164175.0       2  
2500  163948.0       1  
2502  162159.0       1  

[1383 rows x 11 columns]

In [61]:
empresas_reg=empresas.loc[empresas['registro'].notna()]
print(len(empresas_reg))
for i in range(500):
    #comparar nombres
    name_empresa=empresas_reg.iloc[i]['nombre']
    
    try:
        name_encuesta=encuestas20.loc[encuestas20['registro']==empresas_reg.iloc[i]['registro']]['ncr2'].values[0]
        if ((name_encuesta.find(name_empresa) != -1) | ((name_empresa.find(name_encuesta) != -1))):
            # print('coinciden tu')
            x=1
        else:
            print(name_empresa,'|', name_encuesta)
    except:
        # print('registro no encontrado',empresas_reg.iloc[i]['registro'])
        x=2

1636
instituto interamericano de cooperacion para la agricultura (iica) | instituto interamericano de cooperacion para la agricultura-senasica
scayle | scaile
flywheel |  
iconographics | icono graphics s.a. de c.v.
sedesa hospital general topilejo imss bienestar | imss-bienestar hospital general topilejo.
imss unidad de medicina familiar no.160 el vergel | imss unidad de medicina familiar no. 160
imss hgz no. 27 tlatelolco | imss hospital general de zona no. 27
enfoque creativo btl | save the children
imss unidad de medicina familiar no. 180 valle de chalco | imss unidad de medicina familiar 180 valle de chalco
consultorio de optometria | instituto nacional electoral
sep  consejo nacional de fomento educativo (conafe) | secretaria de educacion publica (sep)  consejo nacional de fomento educativo (conafe)
ssa instituto nacional de perinatologia isidro espinoza de los reyes | secretaria de salud instituto nacional de perinatologia isidro espinoza de los reyes
totalplay | total play
bana

In [81]:
empresas.nota.unique()

array([None, 'Empresa verde', 'Empresa sustentable',
       'Se publicita como empresa verde', 'Empresa verde y sustentable',
       'No se encontró la página de la empresa, el egresado vive en Canadá.',
       'No encontré página',
       'Empresa que ofrece productos o servicios en apoyo al medio ambiente.',
       'Empresa Verde', 'Empresa con política ambiental',
       'empresa con política ambiental',
       'Empresa sustentable y que ofrece productos y servicios en apoyo al medio ambiente.',
       'Empresa sustentable. Empresa que ofrece productos o servicios en apoyo al medio ambiente.',
       'Empresa que ofrece servicios en apoyo al medio ambiente.',
       'Empresa sustentable / regenerativa',
       'Se encuentra en instagram, "El hidromiel del Bikingo en vicicleta, hecha por dos químicos chilangos"',
       'Empresas que ofrece servicios en apoyo al medio ambiente.',
       'Empresa que ofrece productos en apoyo al medio ambiente.',
       'Empresa sustentable. Es parte 

In [115]:
empresas_verdes=empresas.loc[empresas['nota'].isin(notas_filtradas)]

In [116]:
empresas_verdes

id usuario                                             nombre  giro  \
1        7      24                                      Grupo PROMESA  None   
3       21      25                                Laboratorios Carnot  None   
6       38      24                                    Jugos del Valle  None   
8       25      22  Instituto Interamericano de Cooperación para l...  None   
14      50      25                                Kueponi Consultoría  None   
...    ...     ...                                                ...   ...   
2454  1449      23                                            Parexel  None   
2481  3423      23                                           Rotoplas  None   
2483  4823      23                                       Tesla México  None   
2495  4843      23                                     Solarig México  None   
2502   722      24  Instituto Nacional de Ecología y Cambio Climát...  None   

     clave_giro                                    giro_especifico  \
1            16  611698 Otros servicios educativos proporcionad...   
3             5  325412 Fabricación de preparaciones farmacéuticas   
6             5  312111 Elaboración de refrescos y otras bebida...   
8            20  93 Actividades legislativas, gubernamentales, ...   
14           12  541620 Servicios de consultoría en medio ambiente   
...         ...                                                ...   
2454         12  541711 Servicios de investigación científica y...   
2481          5  326193 Fabricación de envases y contenedores d...   
2483          5                    31-33 Industrias manufactureras   
2495          3  221113 Generación de electricidad a partir de ...   
2502         12  541712 Servicios de investigación científica y...   

                                                   nota  \
1                                         Empresa verde   
3                                   Empresa sustentable   
6                       Se publicita como empresa verde   
8                                         Empresa verde   
14                          Empresa verde y sustentable   
...                                                 ...   
2454                                Empresa sustentable   
2481                                Empresa sustentable   
2483  Empresa sustentable y que ofrece productos y s...   
2495                        Empresa verde y sustentable   
2502                                  Institución verde   

                    created_at                updated_at  registro  sector  \
1                          NaT 2025-06-19 19:33:23+00:00  161259.0       2   
3                          NaT 2025-01-06 21:28:04+00:00  161278.0       2   
6                          NaT 2025-09-09 16:12:47+00:00  161297.0       2   
8                          NaT 2025-09-09 21:11:19+00:00  161282.0       1   
14                         NaT 2025-06-12 23:38:06+00:00  161310.0       2   
...                        ...                       ...       ...     ...   
2454 2025-04-30 21:24:33+00:00 2025-08-28 21:40:21+00:00       NaN       2   
2481                       NaT 2025-09-01 23:53:37+00:00       NaN       2   
2483 2025-09-02 21:27:36+00:00 2025-09-02 21:27:36+00:00      -1.0       2   
2495                       NaT                       NaT  164139.0       2   
2502                       NaT 2025-09-09 16:08:38+00:00  162159.0       1   

                                      nombre_normalized  
1                                         grupo promesa  
3                                   laboratorios carnot  
6                                       jugos del valle  
8     instituto interamericano de cooperacion para l...  
14                                  kueponi consultoria  
...                                                 ...  
2454                                            parexel  
2481                                           rotoplas  
2483                                       tesla m

In [117]:
#Buscar respuestas20 con empresa verde

palabras_clave = empresas_verdes['nombre_normalized'].unique()
target_respuestas = encuestas20.loc[(encuestas20['ncr2_normalized'].apply(lambda x: any(palabra in x for palabra in palabras_clave)))|(encuestas20['registro'].isin(empresas_verdes['registro'].unique()))]


In [118]:
target_respuestas

registro aplica aplica2                  fec_capt fec_capt2     cuenta  \
5       161736    111    None 2024-12-18 12:03:11+00:00      None  312055129   
60      161498    111    None 2024-11-20 18:24:16+00:00      None  417010072   
74      161631     23    None 2024-12-03 00:00:00+00:00      None  416005332   
108     161308     23    None 2024-11-05 00:00:00+00:00      None  313035126   
120     161635     23    None 2024-12-03 00:00:00+00:00      None  309593614   
...        ...    ...     ...                       ...       ...        ...   
6524    161283     25    None 2024-10-30 00:00:00+00:00      None  314249090   
6526    161300     23    None                       NaT      None  314320414   
6527      3361     23    None                       NaT      None  313183306   
6528    161291     23    None 2024-10-31 00:00:00+00:00      None  313222322   
6532    161292     25    None 2024-10-31 00:00:00+00:00      None  417002743   

                    nombre  paterno    materno telcasa  ... ncr2ext ner12ext  \
5          MONICA GABRIELA    JASSO    ROSALES    None  ...    None     None   
60         CRISTA STEPHANY     MENA  ASPILLAGA    None  ...    None     None   
74    MARIA DE LOS ANGELES   GARCIA     FLORES    None  ...       0     None   
108          MAURICIO IVAN     DIAZ     SEGURA    None  ...       0     None   
120            OMAR ANDRES   BATRES      TREJO    None  ...       0     None   
...                    ...      ...        ...     ...  ...     ...      ...   
6524        MITZI KIMBERLY  ALMARAZ    JIMENEZ    None  ...       0     None   
6526          ALMA CITLALI    ORTIZ  HERNANDEZ    None  ...       0     None   
6527             PERLA LUZ   MORENO     TORRES    None  ...       0        0   
6528                 FRIDA   JUAREZ    RAMIREZ    None  ...       0     None   
6532                VALERY     CRUZ      MEJIA    None  ...       0     None   

     ner15ext  ner18ext  ndr17a  ngr40interes  ngr40otro nfr23bin  empresa_id  \
5        None      None     1.0           NaN       None      2.0        None   
60       None      None     5.0           NaN       None      2.0        None   
74       None      None     5.0           NaN          0      NaN        None   
108      None      None     1.0           NaN          0      NaN        None   
120      None      None     1.0           NaN          0      NaN        None   
...       ...       ...     ...           ...        ...      ...         ...   
6524     None      None     3.0           NaN          0      NaN        None   
6526     None      None     3.0           NaN          0      2.0        None   
6527        0         0     1.0           NaN          0      NaN        None   
6528     None      None     5.0           NaN          0      NaN        None   
6532     None      None     5.0           NaN          0      NaN        None   

                                        ncr2_normalized  
5                                         grupo salinas  
60                                          citibanamex  
74                                        nissan mexico  
108                                           accenture  
120                                         bbva mexico  
...                                                 ...  
6524                                              deriv  
6526                                       krom aduanal  
6527  aosenuma / ensamble alejandria de arte y lectu...  
6528             grupo salinas grupo upax mexa creativa  
6532                                      geodis mexico  

[470 rows x 269 columns]

In [93]:

patron = 'sustentabilidad|sustentable|ecologico|ecologia|energia|reserva|verde'
notas_filtradas = empresas.loc[empresas['nota'].str.contains(patron, case=False, na=False), 'nota'].unique()

print(notas_filtradas)

['Empresa verde' 'Empresa sustentable' 'Se publicita como empresa verde'
 'Empresa verde y sustentable' 'Empresa Verde'
 'Empresa sustentable y que ofrece productos y servicios en apoyo al medio ambiente.'
 'Empresa sustentable. Empresa que ofrece productos o servicios en apoyo al medio ambiente.'
 'Empresa sustentable / regenerativa'
 'Empresa sustentable. Es parte de The Adecco Group y cuenta con reportes de sustentabilidad.'
 'Empresa Sustentable'
 'Empresa sustentable y que que ofrece productos o servicios en apoyo al medio ambiente.'
 'Empresa que en su parte social menciona apoyo al medioambiente, pero no es sustentable en sus procesos.'
 'Empresa sustentable. Egresado cuenta con estudios de posgrado en desarrollo sustentable, es empresa familiar.'
 'Empresa sustentable; además, ofrece productos o servicios en apoyo al medio ambiente.'
 'Empresa sustentable, se encuentra facebook'
 'Según la empresa es sostenible, pero en periódicos se le acusa de "lavado de imagen verde". Financ

In [130]:
sectores_dict={'20' : 'Actividades de gobierno, organismos internacionales y extraterritoriales',
'13' : 'Dirección de corporativos y empresas',
'23' : 'Editorial',
'10' : 'Servicios financieros y de seguros',
'21' : 'Otro',
'5' : 'Industrias manufactureras o de la transformación',
'6' : 'Comercio al por mayor',
'7' : 'Comercio al por menor',
'8' : 'Transporte, correos y almacenamiento',
'9' : 'Información en medios masivos',
'11' : 'Servicio inmobiliario y de alquiler de bienes muebles e intangibles',
'12' : 'Servicios profesionales, científicos y técnicos',
'14' : 'Servicios de apoyo a los negocios, manejo de desecho y servicios de remediación',
'15' : 'Servicios de salud',
'16' : 'Servicios educativos',
'17' : 'Servicios de esparcimiento cultural, deportivos y otros centros recreativos',
'18' : 'Servicios de alojamiento temporal, de preparación de alimentos y bebidas (hotel, restaurante, bar)',
'19' : 'Asociaciones y agrupaciones',
'22' : 'Telecomunicaciones',
'24' : 'Servicios personales',
'25' : 'Servicios de reparacion y mantenimiento',
'1' : 'Agricultura, ganadería, aprovechamiento forestal, caza y pesca',
'2' : 'Minería',
'3' : 'Electricidad, agua y suministro de gas',
'4' : 'Construcción'}

In [132]:
empresas_verdes['clave_giro']=empresas_verdes['clave_giro'].map(sectores_dict)

C:\Users\fgonzalez\AppData\Local\Temp\ipykernel_10044\4102761597.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empresas_verdes['clave_giro']=empresas_verdes['clave_giro'].map(sectores_dict)


In [138]:
empresas_verdes['sector']=empresas_verdes['sector'].map({1 : 'Pública',
2 : 'Privada',
3 : 'Social',})

C:\Users\fgonzalez\AppData\Local\Temp\ipykernel_10044\3143189679.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empresas_verdes['sector']=empresas_verdes['sector'].map({1 : 'Pública',


In [157]:
target_respuestas['nbr2']=target_respuestas['nbr2'].map(dict_carreras)

C:\Users\fgonzalez\AppData\Local\Temp\ipykernel_10044\565564998.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_respuestas['nbr2']=target_respuestas['nbr2'].map(dict_carreras)


In [159]:
target_respuestas['nbr3']=target_respuestas['nbr3'].map(dict_planteles)

C:\Users\fgonzalez\AppData\Local\Temp\ipykernel_10044\579775518.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_respuestas['nbr3']=target_respuestas['nbr3'].map(dict_planteles)


In [ ]:
writer = pd.ExcelWriter('medio_ambiente_2020.xlsx', engine='xlsxwriter')

workbook = writer.book
a_color='#173d83'
#estilos----------------
negro_b = workbook.add_format({
    'bold': 2,
    'border': 0,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':13}) 
header_format = workbook.add_format({
    'bold': True,
    'bg_color': a_color,
    'text_wrap': True,
    'valign': 'top',
    'align': 'center',
    'border_color':'white',
    'font_color': 'white',
    'border': 1,
    'font_size':12})
blue_content = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':10,
    'border_color':a_color})
date_content = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':10,
    'border_color':a_color,
    'num_format': 'dd/mm/yy'})
date_content_bold = workbook.add_format({
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':12,
    'bold': True,
    'num_format': 'dd/mm/yy'})
worksheet = workbook.add_worksheet()
worksheet.merge_range('C2:H3', 'PROGRAMA DE VINCULACION A EGRESADOS UNAM', negro_b)
worksheet.merge_range('C4:H4', 'EMPRESAS VERDES ENCUESTA 2020', negro_b)
worksheet.insert_image("A1", "logoPVE.png",{"x_scale": 0.2, "y_scale": 0.2})
worksheet.merge_range('G6:H6',today, date_content_bold)

worksheet.write('B8','#',header_format)
worksheet.write('C8','Carrera',header_format)
worksheet.write('D8','Plantel',header_format)
worksheet.write('E8','Empresa',header_format)
worksheet.write('F8','Sector de la empresa',header_format)
worksheet.write('G8','Giro de la empresa',header_format)
worksheet.write('H8','Giro específico',header_format)
worksheet.write('I8','Notas de la empresa',header_format)
worksheet.write('J8','Actividad que desempeña el egresado',header_format)
row_index=9
n=1
for i in range(len(target_respuestas)):
    print(n)
    worksheet.write('B'+str(row_index+i),n, blue_content)
    worksheet.write('C'+str(row_index+i),target_respuestas['nbr2'].values[i], blue_content)
    worksheet.write('D'+str(row_index+i),target_respuestas['nbr3'].values[i], blue_content)
    empresa=empresas_verdes.loc[empresas_verdes['registro']==target_respuestas['registro'].values[i]]
    if(len(empresa)==0):
        print(target_respuestas['ncr2_normalized'].values[i])
        target_cadena = target_respuestas['ncr2_normalized'].values[i]
        empresa = empresas_verdes.loc[empresas_verdes['nombre_normalized'].apply(lambda x: x in target_cadena)]

    
    worksheet.write('E'+str(row_index+i),empresa['nombre'].values[0], blue_content)
    worksheet.write('F'+str(row_index+i),empresa['sector'].values[0], blue_content)
    worksheet.write('G'+str(row_index+i),empresa['clave_giro'].values[0], blue_content)
    worksheet.write('H'+str(row_index+i),empresa['giro_especifico'].values[0], blue_content)
    worksheet.write('I'+str(row_index+i),empresa['nota'].values[0], blue_content)
    worksheet.write('J'+str(row_index+i),target_respuestas['ncr12_a'].values[i], blue_content)
    n=n+1
workbook.close() 

1
grupo salinas
2
citibanamex
3
4
5
6
compucom
7
8
grupo salinasgrupo upaxmexa creativa
9
totalplay
10
11
reckitt
12
bbva mexico
13
14
15
16
17
18
at&t mexico
19
qualitas
20
21
baser infraestructura
22
23
24
25
26
aspentech
27
banamex
28
secretaria de infraestructura comunicaciones y transportes
29
30
31
ford mexico
32
sectei puntos de innovacion libertad arte educacion y saberes (pilares) san francisco tultenco
33
ey mexico
34
mota-engil mexico
35
huawei mexico
36
37
38
grupo bimbo
39
40
banorte
41
42
cisco systems de mexico
43
44
solarig mexico
45
banamex
46
ids comercial
47
48
banorte
49
grupo salinas banco ateca
50
daimler truck financial services / alquiladora danys
51
interproteccion
52
53
54
seminario diocesano guadalupano de cuautitlan
55
56
57
58
ericsson telecom s.a. de c.v.
59
60
61
62
63
64
65
66
67
68
69
grupo salinas (corporativo) / black & decker
70
71
walmart de mexico y centroamerica
72
parroquia de san francisco de asis dispensario / trabajador independiente como vend

In [135]:
sectores=pd.read_sql("select * from options where reactivo='ncr3'",cnx)

c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [136]:
for i in range(len(sectores)):
    print("'"+str(sectores['clave'].values[i])+"' : '"+sectores['descripcion'].values[i]+"',")

'1' : 'Pública',
'2' : 'Privada',
'3' : 'Social',


In [139]:
carreras=pd.read_sql("select * from carreras",cnx)

c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [144]:
target_respuestas['nbr3']

5        10
60      600
74        8
108      10
120       8
       ... 
6524    400
6526    400
6527     10
6528     10
6532    400
Name: nbr3, Length: 470, dtype: object

In [151]:
for i in range(len(carreras['clave_carrera'].unique())):
    print("'"+str(carreras['clave_carrera'].unique()[i])+"' : '"+carreras.loc[carreras['clave_carrera']==carreras['clave_carrera'].unique()[i],'carrera'].values[0]+"',")

'105' : 'Diseño Industrial',
'123' : 'Urbanismo',
'401' : 'Artes Visuales',
'405' : 'Comunicación Gráfica',
'406' : 'Diseño Gráfico',
'423' : 'Diseño y Comunicación Visual',
'434' : 'Arte y Diseño',
'435' : 'Cinematografía',
'501' : 'Técnico',
'101' : 'Actuaría',
'104' : 'Ciencias de la Computación',
'106' : 'Física',
'122' : 'Matemáticas',
'127' : 'Ciencias de la Tierra',
'134' : 'Física Biomédica',
'136' : 'Matemáticas Aplicadas',
'201' : 'Biología',
'217' : 'Manejo Sustentable de Zonas Costeras',
'302' : 'Ciencias de la Comunicación (Periodismo)',
'303' : 'Ciencias Políticas y Administración Pública',
'310' : 'Relaciones Internacionales',
'311' : 'Sociología',
'319' : 'Antropología',
'118' : 'Ingeniería Química',
'119' : 'Ingeniería Química y Metalúrgica',
'211' : 'Química',
'212' : 'Química de Alimentos',
'213' : 'Química Farmacéutico Biológica',
'229' : 'Química e Ingeniería en Materiales',
'301' : 'Administración',
'304' : 'Contaduría',
'308' : 'Informática',
'321' : 'Negocios In

In [152]:
dict_carreras={'105' : 'Diseño Industrial',
'123' : 'Urbanismo',
'401' : 'Artes Visuales',
'405' : 'Comunicación Gráfica',
'406' : 'Diseño Gráfico',
'423' : 'Diseño y Comunicación Visual',
'434' : 'Arte y Diseño',
'435' : 'Cinematografía',
'501' : 'Técnico',
'101' : 'Actuaría',
'104' : 'Ciencias de la Computación',
'106' : 'Física',
'122' : 'Matemáticas',
'127' : 'Ciencias de la Tierra',
'134' : 'Física Biomédica',
'136' : 'Matemáticas Aplicadas',
'201' : 'Biología',
'217' : 'Manejo Sustentable de Zonas Costeras',
'302' : 'Ciencias de la Comunicación (Periodismo)',
'303' : 'Ciencias Políticas y Administración Pública',
'310' : 'Relaciones Internacionales',
'311' : 'Sociología',
'319' : 'Antropología',
'118' : 'Ingeniería Química',
'119' : 'Ingeniería Química y Metalúrgica',
'211' : 'Química',
'212' : 'Química de Alimentos',
'213' : 'Química Farmacéutico Biológica',
'229' : 'Química e Ingeniería en Materiales',
'301' : 'Administración',
'304' : 'Contaduría',
'308' : 'Informática',
'321' : 'Negocios Internacionales',
'305' : 'Derecho',
'306' : 'Economía',
'307' : 'Geografía',
'402' : 'Bibliotecología',
'409' : 'Estudios Latinoamericanos',
'411' : 'Filosofía',
'412' : 'Historia',
'414' : 'Lengua y Literaturas Hispánicas',
'415' : 'Lengua y Literaturas Modernas Alemanas',
'416' : 'Lengua y Literaturas Modernas Francesas',
'417' : 'Lengua y Literaturas Modernas Inglesas',
'418' : 'Lengua y Literaturas Modernas Italianas',
'419' : 'Letras Clásicas',
'420' : 'Literatura Dramática y Teatro',
'421' : 'Pedagogía',
'429' : 'Desarrollo y Gestión Interculturales',
'430' : 'Lengua y Literaturas Modernas Portuguesas',
'437' : 'Administración de Archivos y Gestión Documental',
'107' : 'Ingeniería Civil',
'108' : 'Ingeniería de Minas y Metalurgia',
'109' : 'Ingeniería Eléctrica y Electrónica',
'110' : 'Ingeniería en Computación',
'111' : 'Ingeniería en Telecomunicaciones',
'112' : 'Ingeniería Geofísica',
'113' : 'Ingeniería Geológica',
'114' : 'Ingeniería Industrial',
'115' : 'Ingeniería Mecánica',
'116' : 'Ingeniería Mecánica Eléctrica',
'117' : 'Ingeniería Petrolera',
'120' : 'Ingeniería Topográfica y Geodésica',
'124' : 'Ingeniería Mecatrónica',
'125' : 'Ingeniería Geomática',
'135' : 'Ingeniería en Sistemas Biomédicos',
'137' : 'Ingeniería Ambiental',
'139' : 'Ingeniería Aeroespecial',
'206' : 'Investigación Biomédica Básica',
'208' : 'Médico Cirujano',
'221' : 'Fisioterapia',
'224' : 'Ciencia Forense',
'226' : 'Neurociencias',
'231' : 'Ciencia de la Nutrición Humana',
'403' : 'Canto',
'404' : 'Composición',
'407' : 'Educación Musical',
'410' : 'Etnomusicología',
'413' : 'Instrumentista',
'422' : 'Piano',
'436' : 'Teatro y Actuación',
'202' : 'Cirujano Dentista',
'312' : 'Trabajo Social',
'207' : 'Medicina Veterinaria y Zootecnia',
'322' : 'Geografía Aplicada',
'210' : 'Psicología',
'439' : 'Lingüística Aplicada',
'440' : 'Traducción',
'215' : 'Ciencias Genómicas',
'138' : 'Ciencia de Datos',
'128' : 'Ingeniería en Energías Renovables',
'129' : 'Nanotecnología',
'126' : 'Tecnología',
'216' : 'Ciencias Ambientales',
'130' : 'Ingeniería en Telecomunicaciones, Sistemas y Electrónica',
'204' : 'Ingeniería Agrícola',
'205' : 'Ingeniería en Alimentos',
'214' : 'Química Industrial',
'218' : 'Bioquímica Diagnóstica',
'219' : 'Farmacia',
'102' : 'Arquitectura',
'121' : 'Matemáticas Aplicadas y Computación',
'315' : 'Comunicación',
'408' : 'Enseñanza de Inglés',
'424' : 'Enseñanza de Alemán como Lengua Extranjera',
'425' : 'Enseñanza de Español como Lengua Extranjera',
'426' : 'Enseñanza de Francés como Lengua Extranjera',
'427' : 'Enseñanza de Inglés como Lengua Extranjera',
'428' : 'Enseñanza de Italiano como Lengua Extranjera',
'203' : 'Enfermería y Obstetricia',
'209' : 'Optometría',
'220' : 'Enfermería',
'225' : 'Ecología',
'309' : 'Planificación para el Desarrollo Agropecuario',
'316' : 'Comunicación y Periodismo',
'227' : 'Nutriología',
'318' : 'Desarrollo Comunitario para el Envejecimiento',
'222' : 'Odontología',
'223' : 'Ciencias Agrogenómicas',
'313' : 'Economía Industrial',
'314' : 'Administración Agropecuaria',
'320' : 'Desarrollo Territorial',
'323' : 'Turismo y Desarrollo Sostenible',
'131' : 'Geociencias',
'132' : 'Tecnologías para la Información en Ciencias',
'133' : 'Ciencia de Materiales Sustentables',
'230' : 'Ciencias Agroforestales',
'317' : 'Estudios Sociales y Gestión Local',
'431' : 'Geohistoria',
'432' : 'Literatura Intercultural',
'433' : 'Historia del Arte',
'438' : 'Música y Tecnología Artística',
'140' : 'Matemáticas para el Desarrollo',
'228' : 'Órtesis y Prótesis',
'103' : 'Arquitectura del paisaje'}

In [153]:
dict_planteles={'1' : 'Facultad de Arquitectura',
'2' : 'Facultad de Artes y Diseño',
'3' : 'Facultad de Ciencias',
'4' : 'Facultad de Ciencias Políticas y Sociales',
'5' : 'Facultad de Química',
'6' : 'Facultad de Contaduría y Administración',
'7' : 'Facultad de Derecho',
'8' : 'Facultad de Economía',
'10' : 'Facultad de Filosofía y Letras',
'11' : 'Facultad de Ingeniería',
'12' : 'Facultad de Medicina',
'13' : 'Facultad de Música',
'14' : 'Facultad de Odontología',
'15' : 'Escuela Nacional de Trabajo Social',
'16' : 'Facultad de Medicina Veterinaria y Zootecnia',
'17' : 'Escuela Nacional de Ciencias de la Tierra',
'19' : 'Facultad de Psicología',
'42' : 'Escuela Nacional de Lenguas, Lingüística y Traducción',
'44' : 'Escuela Nacional de Artes Cinematográficas',
'66' : 'Centro de Ciencias Genómicas  Instituto de Biotecnología',
'80' : 'Instituto de Investigaciones en Matemáticas Aplicadas y Sistemas',
'90' : 'Instituto de Energías Renovables',
'91' : 'Centro de Nanociencias y Nanotecnología',
'94' : 'Centro de Peninsular en Humanidades y Ciencias Sociales',
'95' : 'Centro de Física Aplicada y Tecnología Avanzada',
'97' : 'Centro de Investigaciones en Ecosistemas',
'100' : 'FES Cuautitlán',
'200' : 'FES Acatlán',
'300' : 'FES Iztacala',
'400' : 'FES Aragón',
'500' : 'FES Zaragoza',
'600' : 'ENES León',
'700' : 'ENES Morelia',
'800' : 'ENES Mérida',
'1000' : 'ENES Juriquilla',
'9' : 'Facultad de Enfermería'}

In [154]:
len(dict_carreras)

135